In [1]:
import argparse

from GeospatialFM.data import get_datasets
from GeospatialFM.models import *

from GeospatialFM.utils import *
from GeospatialFM.data import *
from GeospatialFM.models import *
from GeospatialFM.loss import *

from tqdm import trange
import random
import pandas as pd
from copy import deepcopy

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# exp_name = 'mae_cm_reconall_scratch_allTrain'
# exp_name = 'mae_unidecoder_scratch_allTrain_norwl'
n_spectral_blocks = 1
n_spatial_spectral_blocks = 2
n_spatial_blocks = 9
exp_name = f"mm_vit_{n_spectral_blocks}-{n_spatial_spectral_blocks}-{n_spatial_blocks}_d4_new"
# exp_name = "mae_base_cvit"
device = torch.device('cuda:5')
# device = torch.device('cpu')

In [62]:
args = {'exp_name': exp_name,
        'config_file': 'GeospatialFM/configs/finetune_mm_cvit.yaml',
        'opts': None, 
        'save_config': False,
        'device': 5,
        'finetune_modal': 'multi_modal'}
args = argparse.Namespace(**args)
args.debug = True
args.finetune = True
device = init_distributed_device(args)
cfg, _ = setup(args)
cfg.MODEL.MULTI_MODAL.kwargs.spectral_blocks = n_spectral_blocks
cfg.MODEL.MULTI_MODAL.kwargs.sptial_spectral_blocks = n_spatial_spectral_blocks
cfg.MODEL.load_pretrained_from = 'dir'
cfg.MODEL.MULTI_MODAL.kwargs.low_rank_feature = True
cfg.MODEL.MULTI_MODAL.kwargs.dim_ratio=0.25
args

Namespace(exp_name='mm_vit_1-2-9_d4_new', config_file='GeospatialFM/configs/finetune_mm_cvit.yaml', opts=None, save_config=False, device='cuda:5', finetune_modal='multi_modal', debug=True, finetune=True, distributed=False, world_size=1, rank=0, local_rank=0)

In [63]:
cfg.TRAINER

{'num_train_epochs': 20, 'per_device_train_batch_size': 128, 'per_device_eval_batch_size': 128, 'gradient_accumulation_steps': 4, 'learning_rate': 0.001, 'weight_decay': 0.05, 'dataloader_num_workers': 8, 'dataloader_pin_memory': True, 'dataloader_drop_last': False, 'optim': 'adamw_torch', 'lr_scheduler_type': 'cosine', 'warmup_epochs': 4, 'scheduler_kwargs': None, 'save_strategy': 'epoch', 'evaluation_strategy': 'epoch', 'load_best_model_at_end': True, 'metric_for_best_model': 'accuracy', 'greater_is_better': True, 'logging_dir': './results/logs/BigEarthNet_mm_lr_vit_base_patch16_224_mm_vit_1-2-9_d4_new', 'logging_strategy': 'steps', 'logging_steps': 50, 'report_to': None, 'output_dir': './results/models/BigEarthNet_mm_lr_vit_base_patch16_224_mm_vit_1-2-9_d4_new', 'save_total_limit': 5, 'overwrite_output_dir': False, 'seed': 0, 'fp16': True, 'push_to_hub': False, 'save_frequency': 10, 'ckpt_dir': './results/models/mm_lr_vit_base_patch16_224_mm_vit_1-2-9_d4_new'}

In [64]:
model = construct_downstream_models(cfg, target_modal=args.finetune_modal)

Loading pretrained weights from ./results/models/mm_lr_vit_base_patch16_224_mm_vit_1-2-9_d4_new/final_model.pth...
The model hanles modality by multi_modal...
Targeting multi_modal encoder...
Constructing mm_lr_vit_base_patch16_224 encoder...
Using Multi-Modal LowRank ViT...
Spectral Blocks: 1	Sptial-Spectral Blocks: 2	Spatial Blocks: 9
Using NCD Pooling
Using NCD Pooling
Loading pretrained weights for multi_modal encoder...
Number of parameters for Encoder: 93020064
Constructing linear head...


In [58]:
model.encoder

MultiModalLowRankViTEncoder(
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (ls2): Identity()
      (drop_path2): Identity()
    )
    (1-2): 2 x LowRan

In [59]:
data_idx = 41982
data = get_data(cfg)
sample = data['test'].dataloader.dataset[data_idx]
for key in sample:
    sample[key] = sample[key].unsqueeze(0)

Num Channels: 12
Training Dataset: BigEarthNet
Train Set: 26969	 Val Set: 12372	 Test Set: 125866


In [61]:
sample['radar']


tensor([[[[-1.0272, -0.9353, -0.8286,  ..., -0.6709, -0.7892, -1.0139],
          [-1.0452, -0.9646, -0.8134,  ..., -0.2625, -0.3484, -0.5600],
          [-0.9656, -0.8879, -0.7183,  ..., -0.0288, -0.0823, -0.2611],
          ...,
          [-1.4852, -1.1559, -0.8923,  ..., -1.0711, -0.9008, -0.6822],
          [-1.5152, -1.3264, -1.1180,  ..., -1.1395, -0.9546, -0.7435],
          [-1.4873, -1.5378, -1.4389,  ..., -1.2485, -1.0531, -0.8549]],

         [[ 1.5767,  1.5288,  1.5272,  ...,  1.7101,  1.6934,  1.6822],
          [ 1.7202,  1.7696,  1.8023,  ...,  1.6281,  1.5838,  1.5830],
          [ 1.7928,  1.8721,  1.9104,  ...,  1.5611,  1.5331,  1.5312],
          ...,
          [ 1.8053,  2.0721,  2.2620,  ...,  1.8298,  1.9648,  2.0610],
          [ 1.8167,  2.0844,  2.2758,  ...,  1.8356,  1.9368,  1.9940],
          [ 1.7791,  2.0030,  2.1698,  ...,  1.7676,  1.8097,  1.8005]]]])